# Competition Objective is to detect fraud in transactions; 

## Data


In this competition you are predicting the probability that an online transaction is fraudulent, as denoted by the binary target ```isFraud```.

The data is broken into two files **identity** and **transaction**, which are joined by ```TransactionID```. 

> Note: Not all transactions have corresponding identity information.

**Transaction variables**

- TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
- TransactionAMT: transaction payment amount in USD
- ProductCD: product code, the product for each transaction
- card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
- addr: address
- dist: distance
- P_ and (R__) emaildomain: purchaser and recipient email domain
- C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
- D1-D15: timedelta, such as days between previous transaction, etc.
- M1-M9: match, such as names on card and address, etc.
- Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.

**Categorical Features - Transaction**

- ProductCD
- emaildomain
- card1 - card6
- addr1, addr2
- P_emaildomain
- R_emaildomain
- M1 - M9

**Categorical Features - Identity**

- DeviceType
- DeviceInfo
- id_12 - id_38

**The TransactionDT feature is a timedelta from a given reference datetime (not an actual timestamp).**


# 1. Importation and memory reduction
## 1.1. Importing necessary libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import datetime


# Standard plotly imports
#import plotly.plotly as py
import plotly.graph_objs as go
import ipywidgets
import plotly.express as px
import plotly.tools as tls
from plotly.subplots import make_subplots

from plotly.offline import iplot, init_notebook_mode
#import cufflinks
#import cufflinks as cf
import plotly.figure_factory as ff


# Using plotly + cufflinks in offline mode
init_notebook_mode(connected=True)
#cufflinks.go_offline(connected=True)

# Preprocessing, modelling and evaluating
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold, GridSearchCV
from xgboost import XGBClassifier
import xgboost as xgb
import catboost as cb

## Hyperopt modules
from functools import partial
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_RUNNING


import os
import gc
import time

In [2]:
!jupyter nbextension enable --py widgetsnbextension


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


## 1.2. Importing train datasets

In [3]:
gc.collect()
df_train = pd.read_pickle("/kaggle/input/fraud-detection/df_train_v2.pkl")
df_test = pd.read_pickle("/kaggle/input/fraud-detection/df_test_v2.pkl")
sample_submission = pd.read_csv('/kaggle/input/fraud-detection/sample_submission.csv')

In [4]:
print(df_train.shape)
print(df_test.shape)

(590540, 430)
(506691, 429)


## 1.3. Memory reduction

In [5]:
def resumetable(df):
    n = df.shape[0]
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values  
    summary['Missing %'] = round(summary['Missing'] / n * 100,2)
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values


    return summary


To see the output of the Resume Table, click to see the output 

# 2. Preprocessing

In [6]:
df_train.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_37,id_38,DeviceType,DeviceInfo,_merge,dist1_na,dist2_na,_Weekdays,_Hours,_Days
0,2987000,0,86400,4.226562,W,13926,-inf,150,discover,142,...,T,T,mobile,Others,both,False,True,5,0,2
1,2987001,0,86401,3.367188,W,2755,404,150,mastercard,102,...,F,T,mobile,iOS Device,both,True,True,5,0,2
2,2987002,0,86469,4.078125,W,4663,490,150,visa,166,...,T,T,desktop,Windows,both,False,True,5,0,2
3,2987003,0,86499,3.912109,W,18132,567,150,mastercard,117,...,T,T,desktop,Windows,both,True,True,5,0,2
4,2987004,0,86506,3.912109,H,4497,514,150,mastercard,102,...,T,T,desktop,MacOS,both,True,True,5,0,2


In [7]:
categorical_features = []
for col in df_train.columns.drop('isFraud') :
    if (df_train[col].dtype == 'object' 
        or df_test[col].dtype=='object' 
        or df_train[col].dtype == 'bool') :
        categorical_features.append(col)
categorical_features.extend(['_merge', '_Weekdays',	'_Hours',	'_Days'])

In [8]:
categorical_resume = resumetable(df_train[categorical_features])
categorical_resume

Dataset Shape: (590540, 42)


,Name,dtypes,Missing,Missing %,Uniques,First Value,Second Value,Third Value
0,ProductCD,object,0,0.0,5,W,W,W
1,card1,object,0,0.0,13553,13926,2755,4663
2,card2,object,0,0.0,501,-inf,404,490
3,card3,object,0,0.0,115,150,150,150
4,card4,object,0,0.0,5,discover,mastercard,visa
5,card5,object,0,0.0,120,142,102,166
6,card6,object,0,0.0,5,credit,credit,debit
7,addr1,object,0,0.0,333,315,325,330
8,addr2,object,0,0.0,75,87,87,87
9,P_emaildomain,object,0,0.0,31,gmail.com,gmail.com,outlook.com


In [9]:
for var in categorical_features :
    df_train[var] = df_train[var].apply(str)
    df_test[var] = df_test[var].apply(str)

In [10]:
vega_list = []
for vega in df_train.columns :
    if 'V' in vega: vega_list.append(vega)
        
merged_train_test = pd.concat([df_train[vega_list], df_test[vega_list]], axis =0, ignore_index=True)
for col in vega_list:
    merged_train_test[col] = (preprocessing.minmax_scale(merged_train_test[col], feature_range=(0,1)))

def PCA_change(df, n_components, prefix='PCA_', rand_seed=10):
    pca = PCA(n_components=n_components, random_state=rand_seed)

    principalComponents = pca.fit_transform(df)

    principalComponents = pd.DataFrame(principalComponents)

    principalComponents.rename(columns=lambda x: str(prefix)+str(x), inplace=True)
    
    return principalComponents

merged_train_test = PCA_change(merged_train_test, prefix='PCA_V_', n_components=30)
df_train = pd.concat([df_train, merged_train_test[0:len(df_train)]], axis = 1)
df_test = pd.concat([df_test, merged_train_test[len(df_train):].reset_index(drop= True)], axis = 1)

del merged_train_test
gc.collect()

0

# 2.5 Add feature

In [11]:
import datetime
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
X_train_DT_M = df_train['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
X_train_DT_M = (X_train_DT_M.dt.year-2017)*12 + X_train_DT_M.dt.month 


In [12]:
cat_imp = ['card1', 'card2','addr1', 'P_emaildomain', 'R_emaildomain',
          'M5']
num_imp = ['C13','TransactionAmt','C1','C14','PCA_V_1','C11']

In [13]:
merged_train_test = pd.concat([df_train[cat_imp+num_imp], df_test[cat_imp+num_imp]], axis =0, ignore_index=True)

merged_train_test['uid_1'] = merged_train_test.groupby(['card1', 'card2']).grouper.group_info[0].astype('object')
categorical_features.append('uid_1')

merged_train_test['uid_2'] = merged_train_test.groupby(['uid_1', 'addr1']).grouper.group_info[0].astype('object')
categorical_features.append('uid_2')

merged_train_test['uid_3'] = merged_train_test.groupby(['uid_2', 'P_emaildomain']).grouper.group_info[0].astype('object')
categorical_features.append('uid_3')

merged_train_test['uid_4'] = merged_train_test.groupby(['uid_3', 'R_emaildomain']).grouper.group_info[0].astype('object')
categorical_features.append('uid_4')

merged_train_test['uid_5'] = merged_train_test.groupby(['uid_4', 'M5']).grouper.group_info[0].astype('object')
categorical_features.append('uid_5')

for val_num in num_imp:
    for key in ['uid_1', 'uid_2', 'uid_5', 'uid_4', 'uid_5']:
        merged_train_test[val_num+'_'+key+'_mean'] = merged_train_test[val_num] / merged_train_test.groupby([key])[val_num].transform('mean')
        merged_train_test[val_num+'_'+key+'_std'] = merged_train_test[val_num] / merged_train_test.groupby([key])[val_num].transform('std')

In [14]:
merged_train_test.drop(cat_imp+num_imp, axis = 1, inplace = True)
df_train = pd.concat([df_train, merged_train_test[0:len(df_train)]], axis = 1)
df_test = pd.concat([df_test, merged_train_test[len(df_train):].reset_index(drop= True)], axis = 1)

del merged_train_test
gc.collect()

0

In [15]:
res = df_train.groupby(['uid_1']).agg({'isFraud' : ['count', 'sum']})
res = res.droplevel(0, axis = 1)
res.columns = ['Total_trans', 'Total_fraud']
res.index.name = 'uid_1'
res['fraud_trans'] = res['Total_fraud'] / res['Total_trans']
res.sort_values('Total_fraud', ascending=False).head(20)

,Total_trans,Total_fraud,fraud_trans
uid_1,,,
18242,14112,527.0,0.037344
18383,2188,465.0,0.212523
6972,10332,442.0,0.042780
17736,2067,396.0,0.191582
14239,2813,314.0,0.111625
6008,7918,313.0,0.039530
18693,914,305.0,0.333698
14462,6766,294.0,0.043453
11365,4604,286.0,0.062120


In [16]:
res = df_train.groupby(['uid_2']).agg({'isFraud' : ['count', 'sum']})
res = res.droplevel(0, axis = 1)
res.columns = ['Total_trans', 'Total_fraud']
res.index.name = 'uid_2'
res['N.fraud / N.trans'] = res['Total_fraud'] / res['Total_trans']
res.sort_values('Total_fraud', ascending=False).head(20)

,Total_trans,Total_fraud,N.fraud / N.trans
uid_2,,,
57281,2139,461.0,0.215521
21518,9900,433.0,0.043737
55146,1942,387.0,0.199279
58275,881,304.0,0.345062
43605,2639,297.0,0.112543
35284,4398,274.0,0.062301
57323,1814,257.0,0.141676
15819,834,245.0,0.293765
32124,1260,219.0,0.173810


In [17]:
res = df_train.groupby(['uid_3']).agg({'isFraud' : ['count', 'sum']})
res = res.droplevel(0, axis = 1)
res.columns = ['Total_trans', 'Total_fraud']
res.index.name = 'uid_3'
res['N.fraud / N.trans'] = res['Total_fraud'] / res['Total_trans']
res.sort_values('Total_fraud', ascending=False).head(20)

,Total_trans,Total_fraud,N.fraud / N.trans
uid_3,,,
120081,1061,320.0,0.301602
44347,4093,296.0,0.072319
115683,756,229.0,0.302910
122319,505,204.0,0.403960
90714,1140,183.0,0.160526
120189,808,181.0,0.224010
37826,174,174.0,1.000000
74129,1740,169.0,0.097126
66905,522,148.0,0.283525


In [18]:
res = df_train.groupby(['uid_4']).agg({'isFraud' : ['count', 'sum']})
res = res.droplevel(0, axis = 1)
res.columns = ['Total_trans', 'Total_fraud']
res.index.name = 'uid_4'
res['N.fraud / N.trans'] = res['Total_fraud'] / res['Total_trans']
res.sort_values('Total_fraud', ascending=False).head(20)

,Total_trans,Total_fraud,N.fraud / N.trans
uid_4,,,
152616,1061,320.0,0.301602
55797,4093,296.0,0.072319
146976,756,229.0,0.302910
155444,505,204.0,0.403960
114906,1140,183.0,0.160526
152742,808,181.0,0.224010
47227,174,174.0,1.000000
94294,1740,169.0,0.097126
84977,522,148.0,0.283525


In [19]:
res = df_train.groupby(['uid_5']).agg({'isFraud' : ['count', 'sum']})
res = res.droplevel(0, axis = 1)
res.columns = ['Total_trans', 'Total_fraud']
res.index.name = 'uid_5'
res['N.fraud / N.trans'] = res['Total_fraud'] / res['Total_trans']
res.sort_values('Total_fraud', ascending=False).head(20)

,Total_trans,Total_fraud,N.fraud / N.trans
uid_5,,,
195504,1061,320.0,0.301602
71270,4093,296.0,0.072319
188381,756,229.0,0.302910
198962,505,204.0,0.403960
146765,1140,183.0,0.160526
195630,808,181.0,0.224010
60201,174,174.0,1.000000
120949,1740,169.0,0.097126
108403,522,148.0,0.283525


In [20]:
y_train = df_train['isFraud']
X_train = df_train.drop(['isFraud','TransactionID', 'TransactionDT'], axis = 1)
X_test = df_test.drop(['TransactionID', 'TransactionDT'], axis = 1)
del df_train, df_test
gc.collect()

0

In [21]:
categorical_index = [X_train.columns.get_loc(key) for key in categorical_features]


# 3. Modeling

In [22]:
from sklearn.model_selection import KFold,GroupKFold
from sklearn.metrics import roc_auc_score
from xgboost import plot_importance
from sklearn.metrics import make_scorer

import time
def objective(params):
    time1 = time.time()
    params = {
        'one_hot_max_size': int(params['one_hot_max_size']),
        'random_strength' : round(params['random_strength'],3),
        'max_bin' : int(params['max_bin']),
        'depth': int(params['depth']),
        'l2_leaf_reg': round(params['l2_leaf_reg'],3),
        'learning_rate': round(params['learning_rate'],3),
        'max_leaves': int(params['max_leaves']),
        'min_data_in_leaf': int(params['min_data_in_leaf']),
    }

    print("\n############## New Run ################")
    print(f"params = {params}")
    FOLDS = 6
    count = 1
    gkf = GroupKFold(n_splits=FOLDS)

    y_preds = np.zeros(sample_submission.shape[0])
    y_oof = np.zeros(X_train.shape[0])
    score_mean = 0
    for tr_idx, val_idx in gkf.split(X_train, groups = X_train_DT_M):
        month = X_train_DT_M[val_idx].iloc[0]
        print(f'Fold with holding month: {month}')
        print(f' rows of train = {len(tr_idx)}, rows of holdout ={len(val_idx)}')
        
        X_tr, X_vl = X_train.iloc[tr_idx, :], X_train.iloc[val_idx, :]
        y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
        
        estimator = cb.CatBoostClassifier(eval_metric="AUC",
                                           task_type="GPU",
                                           devices='0:1',
                                           loss_function='Logloss',
                                           iterations = 800,
                                           grow_policy = 'Lossguide',
                                           **params)        
        estimator.fit(
            X_tr,y_tr,
            eval_set=(X_vl, y_vl),
            cat_features=categorical_index,
            use_best_model=True,
            verbose=False)
        
        #y_pred_train = clf.predict_proba(X_vl)[:,1]
        #print(y_pred_train)
        score = make_scorer(roc_auc_score, needs_proba=True)(estimator, X_vl, y_vl)
        # plt.show()
        score_mean += score
        print(f'{count} CV - score: {round(score, 4)}')
        count += 1
    time2 = time.time() - time1
    print(f"Total Time Run: {round(time2 / 60,2)}")
    gc.collect()
    print(f'Mean ROC_AUC: {score_mean / FOLDS}')
    del X_tr, X_vl, y_tr, y_vl, estimator, score
    
    return -(score_mean / FOLDS)


space = {
    'one_hot_max_size': hp.quniform('one_hot_max_size',2,8,1),
    'random_strength': hp.uniform('random_strength', 0.01, 1),
    'max_bin': hp.choice('max_bin', list(range(20, 250, 10))),
    # The maximum depth of a tree, same as GBM.
    # Used to control over-fitting as higher depth will allow model 
    # to learn relations very specific to a particular sample.
    # Should be tuned using CV.
    # Typical values: 3-10
    'depth': hp.quniform('depth', 7, 16, 1),
    
    # reg_lambda: L2 regularization term. L2 encourages smaller weights, this
    # approach can be more useful in tree-models where zeroing 
    # features might not make much sense.
    'l2_leaf_reg': hp.uniform('l2_leaf_reg',  0.1, 4),
    
    # eta: Analogous to learning rate in GBM
    # Makes the model more robust by shrinking the weights on each step
    # Typical final values to be used: 0.01-0.2
    'learning_rate': hp.uniform('learning_rate', 0.07, 0.2),

    # more increases accuracy, but may lead to overfitting.
    # num_leaves: the number of leaf nodes to use. Having a large number 
    # of leaves will improve accuracy, but will also lead to overfitting.
    'max_leaves': hp.choice('max_leaves', list(range(20, 250, 10))),
    
    # specifies the minimum samples per leaf node.
    # the minimum number of samples (data) to group into a leaf. 
    # The parameter can greatly assist with overfitting: larger sample
    # sizes per leaf will reduce overfitting (but may lead to under-fitting).
    'min_data_in_leaf': hp.choice('min_data_in_leaf ', list(range(100, 250, 10))),
    
}

In [23]:
%%time
# Set algoritm parameters
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)

# Print best parameters
best_params = space_eval(space, best)

                                                      
############## New Run ################
params = {'one_hot_max_size': 6, 'random_strength': 0.148, 'max_bin': 240, 'depth': 9, 'l2_leaf_reg': 2.699, 'learning_rate': 0.081, 'max_leaves': 70, 'min_data_in_leaf': 150}
Fold with holding month: 12
 rows of train = 453219, rows of holdout =137321
1 CV - score: 0.9001
Fold with holding month: 15
 rows of train = 488908, rows of holdout =101632
2 CV - score: 0.9381
Fold with holding month: 13
 rows of train = 497955, rows of holdout =92585
3 CV - score: 0.9432
Fold with holding month: 17
 rows of train = 501214, rows of holdout =89326
4 CV - score: 0.9323
Fold with holding month: 14
 rows of train = 504519, rows of holdout =86021
5 CV - score: 0.9403
Fold with holding month: 16
 rows of train = 506885, rows of holdout =83655
6 CV - score: 0.953
Total Time Run: 19.33
Mean ROC_AUC: 0.9345026074907449
                                                                                     
#####

In [24]:
print("BEST PARAMS: ", best_params)

best_params['depth'] = int(best_params['depth'])
best_params['one_hot_max_size'] = int(best_params['one_hot_max_size'])

BEST PARAMS:  {'depth': 9.0, 'l2_leaf_reg': 2.699273276705777, 'learning_rate': 0.08094596338542377, 'max_bin': 240, 'max_leaves': 70, 'min_data_in_leaf': 150, 'one_hot_max_size': 6.0, 'random_strength': 0.14809054761195142}


In [25]:
model = cb.CatBoostClassifier(eval_metric="AUC",
                             task_type="GPU",
                             devices='0:1',
                             loss_function='Logloss',
                             iterations = 2000,
                             grow_policy = 'Lossguide',
                             **best_params)        
model.fit(
    X_train,y_train,
    cat_features = categorical_features,
    verbose=200)

0:	learn: 0.8818343	total: 89.4ms	remaining: 2m 58s
200:	learn: 0.9771169	total: 17.9s	remaining: 2m 39s
400:	learn: 0.9880198	total: 37.5s	remaining: 2m 29s
600:	learn: 0.9934689	total: 55.8s	remaining: 2m 9s
800:	learn: 0.9964118	total: 1m 14s	remaining: 1m 51s
1000:	learn: 0.9980077	total: 1m 33s	remaining: 1m 32s
1200:	learn: 0.9989232	total: 1m 53s	remaining: 1m 15s
1400:	learn: 0.9994287	total: 2m 11s	remaining: 56.4s
1600:	learn: 0.9997060	total: 2m 30s	remaining: 37.5s
1800:	learn: 0.9998507	total: 2m 51s	remaining: 18.9s
1999:	learn: 0.9999222	total: 3m 9s	remaining: 0us


In [26]:
sample_submission['isFraud'] = 1 - model.predict_proba(X_test)
sample_submission.head()
sample_submission.to_csv('catboost_new_tuned.csv', index=False)

In [27]:
del model
gc.collect()

0

In [28]:
FOLDS = 6
count = 1
gkf = GroupKFold(n_splits=FOLDS)

predictions = np.zeros(sample_submission.shape[0])
oof_preds = np.zeros(sample_submission.shape[0])
oof = np.zeros(X_train.shape[0])
score_mean = 0
for tr_idx, val_idx in gkf.split(X_train, groups = X_train_DT_M):
    month = X_train_DT_M[val_idx].iloc[0]
    print(f'Fold with holding month: {month}')
    print(f' rows of train = {len(tr_idx)}, rows of holdout ={len(val_idx)}')

    X_tr, X_vl = X_train.iloc[tr_idx, :], X_train.iloc[val_idx, :]
    y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]

    estimator = cb.CatBoostClassifier(eval_metric="AUC",
                                       task_type="GPU",
                                       devices='0:1',
                                       loss_function='Logloss',
                                       iterations = 2000,
                                       grow_policy = 'Lossguide',
                                       **best_params)        
    estimator.fit(
        X_tr,y_tr,
        eval_set=(X_vl, y_vl),
        cat_features=categorical_index,
        use_best_model=True,
        verbose=500)
    
    oof_preds = estimator.predict_proba(X_vl)[:,1]
    oof[val_idx] = (oof_preds - oof_preds.min())/(oof_preds.max() - oof_preds.min())
    predictions += estimator.predict_proba(X_test)[:,1]/FOLDS
    del estimator,  X_tr, X_vl, y_tr, y_vl
    gc.collect()
        
print('OOF AUC:', roc_auc_score(y_train, oof))

Fold with holding month: 12
 rows of train = 453219, rows of holdout =137321
0:	learn: 0.8842098	test: 0.7899301	best: 0.7899301 (0)	total: 94.2ms	remaining: 3m 8s
500:	learn: 0.9940544	test: 0.9011289	best: 0.9012297 (496)	total: 46.3s	remaining: 2m 18s
1000:	learn: 0.9991149	test: 0.8984643	best: 0.9013225 (502)	total: 1m 32s	remaining: 1m 32s
1500:	learn: 0.9998846	test: 0.8978925	best: 0.9013225 (502)	total: 2m 19s	remaining: 46.5s
1999:	learn: 0.9999881	test: 0.8972766	best: 0.9013225 (502)	total: 3m 5s	remaining: 0us
bestTest = 0.9013225138
bestIteration = 502
Shrink model to first 503 iterations.
Fold with holding month: 15
 rows of train = 488908, rows of holdout =101632
0:	learn: 0.8794120	test: 0.8242052	best: 0.8242052 (0)	total: 87.9ms	remaining: 2m 55s
500:	learn: 0.9933802	test: 0.9358808	best: 0.9361979 (371)	total: 45.1s	remaining: 2m 15s
1000:	learn: 0.9990019	test: 0.9360102	best: 0.9363837 (682)	total: 1m 32s	remaining: 1m 32s
1500:	learn: 0.9998639	test: 0.9354709	b

In [29]:
sample_submission['isFraud'] = predictions
sample_submission.head()
sample_submission.to_csv('catboost_new_tuned_cv.csv', index=False)